In [ ]:
with open("/Users/abhishek/.profile", "r") as env_file:
    for line in env_file:
        if line.strip() != "":
            print(f"Line Data: '{line.strip()}'")
            key, value = line.strip().split(sep="=")
            os.environ[key] = value.replace('"','')

In [ ]:
from argparse import Namespace
from enum import Enum, auto
from logging import debug
import os
from typing import Dict, List
import yaml
from ccloud.core import create_ccloud_request, execute_ccloud_request, get_http_connection, generate_iso8601_dt_intervals
from ccloud.model import CCMEReq_Granularity
from data_processing.metrics_processing import metrics_dataframe
import helpers
from config_parser import try_parse_config_file, locate_storage_path, DirType
import datetime
import pandas as pd
import csv


core_config = try_parse_config_file(config_yaml_path="/Users/abhishek/Non-Cloud/repo/ccloud-chargeback-helper/config/config.yaml")

In [ ]:
storage_path = locate_storage_path(dir_type=[DirType.MetricsData, DirType.BillingsData, DirType.OutputData])

In [ ]:
intervals = generate_iso8601_dt_intervals(CCMEReq_Granularity.P1D.name, intervals=3)
print(intervals)

In [ ]:
new_req = create_ccloud_request(request=core_config["configs"]["connection"][0]["requests"][0], intervals=3)
print(new_req)

In [ ]:
connection = get_http_connection(ccloud_details=core_config["configs"]["connection"][0]["ccloud_details"])


In [ ]:
resp_code, resp_body = execute_ccloud_request(
    ccloud_url="https://api.telemetry.confluent.cloud/v2/metrics/cloud/query", auth=connection, payload=new_req, timeout=200
)
print(resp_body)

In [ ]:
curr_df = metrics_dataframe(aggregation_metric_name=new_req["aggregations"][0]["metric"], metrics_output=resp_body)

In [ ]:
curr_df.print_sample_df()

In [ ]:
# this is for the timestamp based pivoted dataset
from csv import QUOTE_NONNUMERIC

trial_out_basepath = storage_path[DirType.MetricsData]
# dt_range = curr_df.get_dataset(ds_name="pivoted_on_timestamp")[1].index.unique().normalize().unique()
dt_range = curr_df.get_dataset(ds_name="pivoted_on_timestamp")[1].index.unique().normalize().unique()
dates_arr = dt_range.date
print(dt_range)
print(dates_arr)

for dt_data, d_data in zip(dt_range, dates_arr):
    print(d_data)
    print(dt_data)
    new_item = dt_data + datetime.timedelta(days=1)
    print(new_item)
    print(type(new_item))
    ds = curr_df.get_dataset(ds_name="pivoted_on_timestamp")[1]
    subset = ds[(ds.index >= dt_data) & (ds.index < new_item)]
    out_path = os.path.join(trial_out_basepath, f"{d_data}_pivoted_on_timestamp.csv")
    subset.to_csv(out_path, quoting=QUOTE_NONNUMERIC)
    

In [ ]:
ds = curr_df.get_dataset(ds_name="metricsapi_representation")[1]
ts_range = ds["timestamp"].dt.normalize().unique()
print(ts_range)
dt_range = ts_range.date
print(dt_range)
print(type(ts_range))
print(type(dt_range))


In [ ]:
import csv
for ts_date, dt_str in zip(ts_range, dt_range):
    lt_date = ts_date + datetime.timedelta(days=1)
    print(ts_date, dt_str, lt_date)
    subset = ds[(ds["timestamp"] >= ts_date) & (ds["timestamp"] < lt_date)]
    print(subset.info())
    out_path = os.path.join(trial_out_basepath, f"{dt_str}_metricsapi_representation.csv")
    subset.to_csv(out_path, index=False, quoting=csv.QUOTE_NONNUMERIC)
